In [1]:
### Environment setup
from pystac_client import Client
import planetary_computer as pc
import os

# Set the environment variable PC_SDK_SUBSCRIPTION_KEY, or set it here.
# The Hub sets PC_SDK_SUBSCRIPTION_KEY automatically.
# pc.settings.set_subscription_key(<YOUR API Key>)
env_vars = !cat /content/.env

for var in env_vars:
    key, value = var.split(' = ')
    os.environ[key] = value

In [2]:
import pandas as pd
import fsspec
import numpy as np
import geopandas as gpd
import rasterio
from rasterio import windows
from rasterio import features
from rasterio import warp

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
#experiments with dask
import dask.dataframe as dd

container = 'itv-data'

storage_options={'account_name':os.environ['ACCOUNT_NAME'],\
                 'account_key':os.environ['BLOB_KEY']}
fs = fsspec.filesystem('az', account_name=storage_options['account_name'], account_key=storage_options['account_key'])
station_url = f'az://{container}/itv_station_metadata.csv'
station_df = dd.read_csv(station_url, storage_options=storage_options)                
#df = dd.read_csv(station_url, storage_options=storage_options)
#ddf = dg.from_dask_dataframe(df).set_geometry(dg.points_from_xy(ddf, 'Latitude', 'Longitude'))

# Here we preprocessing of the data for standardized ingestion

In [8]:
for site_no in station_df.site_no:
    station_url = f'az://{container}/{site_no}.csv'
    site_df = pd.read_csv(station_url,\
                          storage_options=storage_options,\
                          delimiter=',')

    site_df['Date-Time']=pd.to_datetime(site_df['Campaign Date'], \
                                       format='%d/%m/%Y')

    write_filename = f'az://{container}/stations/{site_no}.csv'
    site_df.to_csv(write_filename, storage_options=storage_options,\
                  index=False)